In [ ]:
%config Completer.use_jedi = False

In [ ]:
import pandas as pd

data_path = '/kaggle/input/bike-sharing-demand/'

train = pd.read_csv(data_path + 'train.csv')
test= pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

### 6.4.1 피처 엔지니어링

* 이상치 제거

In [ ]:
# 훈련 데이터에서 weather가 4가 아닌 데이터만 추출
train = train[train['weather'] != 4]

* 데이터 합치기

In [ ]:
all_data_tmp = pd.concat([train, test])
all_data_tmp

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

In [ ]:
from datetime import datetime

#날짜 피처 생성
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
#연도 피처 생성
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
# 월 피처 생성
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
# 시 피처 생성
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])
#요일 피처 생성
all_data['weekday'] = all_data['date'].apply(lambda dateString: datetime.strptime(dateString, '%Y-%m-%d').weekday())

* 필요 없는 피처 제거

In [ ]:
drop_features = ['casual', 'registered', 'datetime', 'date', 'windspeed', 'month']
all_data = all_data.drop(drop_features, axis=1)

**피처 선택**  
모델링 시 데이터의 특징을 잘 나타내는 주요 피처만 선택하는 작업을 **피처 선택**(feature selection)이라고 한다. 피처 선택은 머신러닝 모델 성능에 큰 영향을 준다. 타깃값 예측과 관련 없는 피처가 많다면 오히려 예측 성능이 떨어진다. 피처 선택 방법에는 정답이 없다. 어떤 피처를 선택해야 성능이 가장 좋을지 바로 알 방법은 없다. 탐색적 데이터 분석, 피처 중요도(feature importance), 상관관계 매트릭스 등을 활용해 종합적으로 판단해야 한다.

* 데이터 나누기

In [ ]:
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count']

In [ ]:
X_train.head()

### 6.4.2 평가지표 계산 함수 작성

In [ ]:
import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    # 지수 변환
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)
    
    # 로그변환 후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))
    
    #RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred) ** 2))
    return output
    

### 6.4.3 모델 훈련

In [ ]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

In [ ]:

log_y = np.log(y) #타깃값 로그 변환
linear_reg_model.fit(X_train, log_y) #모델 훈련

### 6.4.4 모델 성능 검증

In [ ]:
preds = linear_reg_model.predict(X_train)

In [ ]:
print(f'선형 회귀의 rmsle 값: {rmsle(log_y, preds, True): .4f}')

### 6.4.5 예측 및 결과 제출

In [ ]:
linearreg_preds = linear_reg_model.predict(X_test) #테스트 데이터로 예측
submission['count'] = np.exp(linearreg_preds) # 지수 변환
submission.to_csv('submission.csv', index=False)